In [1]:
!conda activate tf_gpu

In [2]:
# !pip install keras~=2.6.0

In [3]:
#importando pacotes
import math
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
#Limitando o uso da GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

In [5]:
#Padronizando as imagens

# Importando dataset 
(x_treino, y_treino), (x_teste, y_teste) = fashion_mnist.load_data()

# Normalizando as imagens
x_treino, x_teste = x_treino/255., x_teste/255.

# Ajusta o shape de x de 28x28 para 784
x_treino = tf.reshape(x_treino, shape = (-1, 784))
x_teste  = tf.reshape(x_teste, shape = (-1, 784))

In [6]:
### Construindo o Modelo

# Definindo os pesos, variáveis W e b

# Inicializando os Coeficientes de Forma Randômica com Distribuição Normal
pesos = tf.Variable(tf.random.normal(shape = (784, 10), dtype = tf.float64))
vieses  = tf.Variable(tf.random.normal(shape = (10,), dtype = tf.float64))


# Função para o cálculo do resultado da regressão logísitica
# g(y) = β(x) + βo  
def logistic_regression(x):
    lr = tf.add(tf.matmul(x, pesos), vieses)
    return lr


# Minimizando o erro usando cross entropy (Função de Custo)
#função de erro de entropia cruzada
def cross_entropy(y_true, y_pred):
    y_true = tf.one_hot(y_true, 10)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits = y_pred)
    return tf.reduce_mean(loss)


# Otimizando a Cost Function
# Minimizando função de custo usando o algoritmo de otimização de descida de gradiente:
def grad(x, y):
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [pesos, vieses])


# Hiperparâmetros
n_batches = 10000
learning_rate = 0.01
batch_size = 128


# Cria o otimizador usando SGD (Stochastic Gradient Descent)
optimizer = tf.optimizers.SGD(learning_rate)


# Função para o cálculo da Acurácia
def accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, dtype = tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis = 1), dtype = tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype = tf.float32))

In [7]:
# Preparando batches de dados de treino
dataset_treino = tf.data.Dataset.from_tensor_slices((x_treino, y_treino))
dataset_treino = dataset_treino.repeat().shuffle(x_treino.shape[0]).batch(batch_size)

In [8]:
# Ciclo de treinamento
print ("\nIniciando o Treinamento!")

for batch_numb, (batch_xs_treino, batch_ys_treino) in enumerate(dataset_treino.take(n_batches), 1):

    # Calcula os gradientes
    gradientes = grad(batch_xs_treino, batch_ys_treino)

    # Otimiza os pesos com o valor do gradiente
    optimizer.apply_gradients(zip(gradientes, [pesos, vieses]))

    # Faz uma previsão
    y_pred = logistic_regression(batch_xs_treino)

    # Calcula o erro
    loss = cross_entropy(batch_ys_treino, y_pred)

    # Calcula a acurácia
    acc = accuracy(batch_ys_treino, y_pred)

    # Print
    print("Número do Batch: %i, Erro do Modelo: %f, Acurácia em Treino: %f" % (batch_numb, loss, acc))

print ("\nTreinamento concluído!")


Iniciando o Treinamento!
Número do Batch: 1, Erro do Modelo: 18.027057, Acurácia em Treino: 0.062500
Número do Batch: 2, Erro do Modelo: 15.792562, Acurácia em Treino: 0.171875
Número do Batch: 3, Erro do Modelo: 16.917577, Acurácia em Treino: 0.125000
Número do Batch: 4, Erro do Modelo: 16.587021, Acurácia em Treino: 0.101562
Número do Batch: 5, Erro do Modelo: 17.294258, Acurácia em Treino: 0.101562
Número do Batch: 6, Erro do Modelo: 16.068490, Acurácia em Treino: 0.117188
Número do Batch: 7, Erro do Modelo: 15.922869, Acurácia em Treino: 0.140625
Número do Batch: 8, Erro do Modelo: 13.902186, Acurácia em Treino: 0.164062
Número do Batch: 9, Erro do Modelo: 15.424991, Acurácia em Treino: 0.148438
Número do Batch: 10, Erro do Modelo: 16.158622, Acurácia em Treino: 0.085938
Número do Batch: 11, Erro do Modelo: 13.818003, Acurácia em Treino: 0.125000
Número do Batch: 12, Erro do Modelo: 14.204039, Acurácia em Treino: 0.117188
Número do Batch: 13, Erro do Modelo: 12.823003, Acurácia em

Número do Batch: 139, Erro do Modelo: 6.440174, Acurácia em Treino: 0.195312
Número do Batch: 140, Erro do Modelo: 6.368928, Acurácia em Treino: 0.250000
Número do Batch: 141, Erro do Modelo: 5.819756, Acurácia em Treino: 0.203125
Número do Batch: 142, Erro do Modelo: 7.068844, Acurácia em Treino: 0.179688
Número do Batch: 143, Erro do Modelo: 5.822461, Acurácia em Treino: 0.250000
Número do Batch: 144, Erro do Modelo: 6.318967, Acurácia em Treino: 0.226562
Número do Batch: 145, Erro do Modelo: 5.872190, Acurácia em Treino: 0.179688
Número do Batch: 146, Erro do Modelo: 6.176635, Acurácia em Treino: 0.226562
Número do Batch: 147, Erro do Modelo: 6.108148, Acurácia em Treino: 0.218750
Número do Batch: 148, Erro do Modelo: 6.173946, Acurácia em Treino: 0.250000
Número do Batch: 149, Erro do Modelo: 6.208176, Acurácia em Treino: 0.226562
Número do Batch: 150, Erro do Modelo: 5.564552, Acurácia em Treino: 0.210938
Número do Batch: 151, Erro do Modelo: 6.635073, Acurácia em Treino: 0.218750

Número do Batch: 254, Erro do Modelo: 5.375962, Acurácia em Treino: 0.250000
Número do Batch: 255, Erro do Modelo: 4.946868, Acurácia em Treino: 0.296875
Número do Batch: 256, Erro do Modelo: 4.785258, Acurácia em Treino: 0.351562
Número do Batch: 257, Erro do Modelo: 5.181776, Acurácia em Treino: 0.250000
Número do Batch: 258, Erro do Modelo: 4.957578, Acurácia em Treino: 0.289062
Número do Batch: 259, Erro do Modelo: 4.947800, Acurácia em Treino: 0.289062
Número do Batch: 260, Erro do Modelo: 5.140757, Acurácia em Treino: 0.242188
Número do Batch: 261, Erro do Modelo: 4.240686, Acurácia em Treino: 0.289062
Número do Batch: 262, Erro do Modelo: 4.925524, Acurácia em Treino: 0.296875
Número do Batch: 263, Erro do Modelo: 4.708228, Acurácia em Treino: 0.335938
Número do Batch: 264, Erro do Modelo: 4.501256, Acurácia em Treino: 0.328125
Número do Batch: 265, Erro do Modelo: 3.841125, Acurácia em Treino: 0.359375
Número do Batch: 266, Erro do Modelo: 4.811780, Acurácia em Treino: 0.359375

Número do Batch: 362, Erro do Modelo: 4.113323, Acurácia em Treino: 0.382812
Número do Batch: 363, Erro do Modelo: 3.897131, Acurácia em Treino: 0.429688
Número do Batch: 364, Erro do Modelo: 3.947798, Acurácia em Treino: 0.375000
Número do Batch: 365, Erro do Modelo: 4.135062, Acurácia em Treino: 0.375000
Número do Batch: 366, Erro do Modelo: 3.650888, Acurácia em Treino: 0.429688
Número do Batch: 367, Erro do Modelo: 3.956905, Acurácia em Treino: 0.351562
Número do Batch: 368, Erro do Modelo: 3.943562, Acurácia em Treino: 0.335938
Número do Batch: 369, Erro do Modelo: 4.506331, Acurácia em Treino: 0.312500
Número do Batch: 370, Erro do Modelo: 3.988203, Acurácia em Treino: 0.359375
Número do Batch: 371, Erro do Modelo: 3.823974, Acurácia em Treino: 0.390625
Número do Batch: 372, Erro do Modelo: 4.115259, Acurácia em Treino: 0.328125
Número do Batch: 373, Erro do Modelo: 4.497908, Acurácia em Treino: 0.351562
Número do Batch: 374, Erro do Modelo: 3.914744, Acurácia em Treino: 0.359375

Número do Batch: 469, Erro do Modelo: 3.791467, Acurácia em Treino: 0.335938
Número do Batch: 470, Erro do Modelo: 3.550119, Acurácia em Treino: 0.375000
Número do Batch: 471, Erro do Modelo: 4.629479, Acurácia em Treino: 0.351562
Número do Batch: 472, Erro do Modelo: 3.358081, Acurácia em Treino: 0.375000
Número do Batch: 473, Erro do Modelo: 3.723144, Acurácia em Treino: 0.382812
Número do Batch: 474, Erro do Modelo: 4.339817, Acurácia em Treino: 0.406250
Número do Batch: 475, Erro do Modelo: 4.141027, Acurácia em Treino: 0.453125
Número do Batch: 476, Erro do Modelo: 3.546709, Acurácia em Treino: 0.484375
Número do Batch: 477, Erro do Modelo: 3.771083, Acurácia em Treino: 0.367188
Número do Batch: 478, Erro do Modelo: 3.754169, Acurácia em Treino: 0.382812
Número do Batch: 479, Erro do Modelo: 3.673652, Acurácia em Treino: 0.390625
Número do Batch: 480, Erro do Modelo: 3.496264, Acurácia em Treino: 0.437500
Número do Batch: 481, Erro do Modelo: 3.710190, Acurácia em Treino: 0.382812

...

Número do Batch: 9485, Erro do Modelo: 1.378216, Acurácia em Treino: 0.796875
Número do Batch: 9486, Erro do Modelo: 1.111390, Acurácia em Treino: 0.742188
Número do Batch: 9487, Erro do Modelo: 1.165577, Acurácia em Treino: 0.750000
Número do Batch: 9488, Erro do Modelo: 1.062292, Acurácia em Treino: 0.765625
Número do Batch: 9489, Erro do Modelo: 1.077274, Acurácia em Treino: 0.718750
Número do Batch: 9490, Erro do Modelo: 1.644861, Acurácia em Treino: 0.625000
Número do Batch: 9491, Erro do Modelo: 1.343876, Acurácia em Treino: 0.679688
Número do Batch: 9492, Erro do Modelo: 1.006377, Acurácia em Treino: 0.742188
Número do Batch: 9493, Erro do Modelo: 1.157462, Acurácia em Treino: 0.765625
Número do Batch: 9494, Erro do Modelo: 1.079662, Acurácia em Treino: 0.734375
Número do Batch: 9495, Erro do Modelo: 1.242353, Acurácia em Treino: 0.734375
Número do Batch: 9496, Erro do Modelo: 1.219167, Acurácia em Treino: 0.718750
Número do Batch: 9497, Erro do Modelo: 1.666175, Acurácia em Tre

Número do Batch: 9595, Erro do Modelo: 1.207696, Acurácia em Treino: 0.703125
Número do Batch: 9596, Erro do Modelo: 1.000639, Acurácia em Treino: 0.726562
Número do Batch: 9597, Erro do Modelo: 1.308939, Acurácia em Treino: 0.710938
Número do Batch: 9598, Erro do Modelo: 1.146512, Acurácia em Treino: 0.765625
Número do Batch: 9599, Erro do Modelo: 0.987671, Acurácia em Treino: 0.742188
Número do Batch: 9600, Erro do Modelo: 1.437901, Acurácia em Treino: 0.710938
Número do Batch: 9601, Erro do Modelo: 1.576070, Acurácia em Treino: 0.648438
Número do Batch: 9602, Erro do Modelo: 0.877772, Acurácia em Treino: 0.796875
Número do Batch: 9603, Erro do Modelo: 1.118726, Acurácia em Treino: 0.773438
Número do Batch: 9604, Erro do Modelo: 1.722423, Acurácia em Treino: 0.687500
Número do Batch: 9605, Erro do Modelo: 1.711282, Acurácia em Treino: 0.671875
Número do Batch: 9606, Erro do Modelo: 1.218763, Acurácia em Treino: 0.773438
Número do Batch: 9607, Erro do Modelo: 1.202807, Acurácia em Tre

Número do Batch: 9705, Erro do Modelo: 1.209761, Acurácia em Treino: 0.750000
Número do Batch: 9706, Erro do Modelo: 1.222961, Acurácia em Treino: 0.726562
Número do Batch: 9707, Erro do Modelo: 1.220470, Acurácia em Treino: 0.734375
Número do Batch: 9708, Erro do Modelo: 2.029535, Acurácia em Treino: 0.671875
Número do Batch: 9709, Erro do Modelo: 0.994239, Acurácia em Treino: 0.781250
Número do Batch: 9710, Erro do Modelo: 0.998534, Acurácia em Treino: 0.757812
Número do Batch: 9711, Erro do Modelo: 1.227901, Acurácia em Treino: 0.789062
Número do Batch: 9712, Erro do Modelo: 1.639490, Acurácia em Treino: 0.687500
Número do Batch: 9713, Erro do Modelo: 1.281826, Acurácia em Treino: 0.671875
Número do Batch: 9714, Erro do Modelo: 1.224928, Acurácia em Treino: 0.734375
Número do Batch: 9715, Erro do Modelo: 1.052667, Acurácia em Treino: 0.703125
Número do Batch: 9716, Erro do Modelo: 1.137122, Acurácia em Treino: 0.726562
Número do Batch: 9717, Erro do Modelo: 1.516001, Acurácia em Tre

Número do Batch: 9833, Erro do Modelo: 1.360201, Acurácia em Treino: 0.734375
Número do Batch: 9834, Erro do Modelo: 1.358080, Acurácia em Treino: 0.710938
Número do Batch: 9835, Erro do Modelo: 1.200943, Acurácia em Treino: 0.750000
Número do Batch: 9836, Erro do Modelo: 1.293779, Acurácia em Treino: 0.718750
Número do Batch: 9837, Erro do Modelo: 1.457972, Acurácia em Treino: 0.695312
Número do Batch: 9838, Erro do Modelo: 1.505457, Acurácia em Treino: 0.710938
Número do Batch: 9839, Erro do Modelo: 0.955588, Acurácia em Treino: 0.742188
Número do Batch: 9840, Erro do Modelo: 1.678767, Acurácia em Treino: 0.734375
Número do Batch: 9841, Erro do Modelo: 1.669307, Acurácia em Treino: 0.687500
Número do Batch: 9842, Erro do Modelo: 1.343878, Acurácia em Treino: 0.664062
Número do Batch: 9843, Erro do Modelo: 1.064761, Acurácia em Treino: 0.750000
Número do Batch: 9844, Erro do Modelo: 1.059408, Acurácia em Treino: 0.750000
Número do Batch: 9845, Erro do Modelo: 0.933638, Acurácia em Tre

Número do Batch: 9969, Erro do Modelo: 1.287173, Acurácia em Treino: 0.765625
Número do Batch: 9970, Erro do Modelo: 1.258143, Acurácia em Treino: 0.757812
Número do Batch: 9971, Erro do Modelo: 1.354226, Acurácia em Treino: 0.671875
Número do Batch: 9972, Erro do Modelo: 1.236349, Acurácia em Treino: 0.710938
Número do Batch: 9973, Erro do Modelo: 1.145758, Acurácia em Treino: 0.695312
Número do Batch: 9974, Erro do Modelo: 1.208717, Acurácia em Treino: 0.695312
Número do Batch: 9975, Erro do Modelo: 1.925342, Acurácia em Treino: 0.625000
Número do Batch: 9976, Erro do Modelo: 1.402749, Acurácia em Treino: 0.664062
Número do Batch: 9977, Erro do Modelo: 1.524215, Acurácia em Treino: 0.679688
Número do Batch: 9978, Erro do Modelo: 1.298265, Acurácia em Treino: 0.742188
Número do Batch: 9979, Erro do Modelo: 1.227092, Acurácia em Treino: 0.734375
Número do Batch: 9980, Erro do Modelo: 0.954799, Acurácia em Treino: 0.773438
Número do Batch: 9981, Erro do Modelo: 1.092387, Acurácia em Tre

In [10]:
# Testando o Modelo

# Preparando os dados de teste
dataset_teste = tf.data.Dataset.from_tensor_slices((x_teste, y_teste))
dataset_teste = dataset_teste.repeat().shuffle(x_teste.shape[0]).batch(batch_size)

In [11]:
print ("\nIniciando a Avaliação com Dados de Teste. Por favor aguarde!")

# Loop pelos dados de teste, previsões e cálculo da acurácia
for batch_numb, (batch_xs_teste, batch_ys_teste) in enumerate(dataset_teste.take(n_batches), 1):
    y_pred = logistic_regression(batch_xs_teste)
    acc = accuracy(batch_ys_teste, y_pred)
    acuracia = tf.reduce_mean(tf.cast(acc, tf.float64))

print("\nAcurácia em Teste: %f" % acuracia)


Iniciando a Avaliação com Dados de Teste. Por favor aguarde!

Acurácia em Teste: 0.773438


In [13]:
# Fazendo previsões

# Obtendo os dados de algumas imagens
dataset_teste = tf.data.Dataset.from_tensor_slices((x_teste, y_teste))
dataset_teste = dataset_teste.repeat().shuffle(x_teste.shape[0]).batch(1)

print("\nFazendo Previsão de Uma Imagem:")

for batch_numb, (batch_xs, batch_ys) in enumerate(dataset_teste.take(1), 1):
    # print("\nImagem:", batch_xs)
    print("\nClasse Real:", batch_ys)
    y_pred = tf.math.argmax(logistic_regression(batch_xs), axis = 1)
    # y_pred = logistic_regression(batch_xs)
    print("Classe Prevista:", y_pred)


Fazendo Previsão de Uma Imagem:

Classe Real: tf.Tensor([8], shape=(1,), dtype=uint8)
Classe Prevista: tf.Tensor([8], shape=(1,), dtype=int64)
